# Hot start analysis

Notebook to analyse difference between hot start and the final solution.

Import required functions and import, or declare, constants:

In [ ]:
from modules.helper_functions_tsp import (hot_start, 
                                         find_problem_size,
                                         find_distances_array,
                                         hot_start_list_to_string,
                                         cost_fn_fact,
                                         convert_integer_to_binary_list,
                                         convert_bit_string_to_cycle,
                                         )   

import pandas as pd
from pathlib import Path
import numpy as np
from typing import Callable

from modules.config import (DECODING_FORMULATION, 
                            GRAY, 
                            RESULTS_DIR
                            )
START_LOCATION = 4
END_LOCATION = 12
FILENAME = 'hot_start_hamming_distance.csv'

Declare dataframe:

In [2]:
print(f'Running experiment with {DECODING_FORMULATION} decoding formulation and {GRAY} gray code.')
df = pd.DataFrame(columns=['locations', 
                           'qubits', 
                           'best dist',
                           'hot start dist',
                           'quality',
                           'lowest hamming dist',])

Running experiment with original decoding formulation and False gray code.


Define required function modules:

In [ ]:
def find_hot_start_distance(distance_array: np.array, 
                            locations:int, 
                            cost_fn: Callable
                            ) -> tuple:
    """find the hot start distance and the corresponding binary list"""
    hot_start_list = hot_start(distance_array, locations)
    print(f'The hot start location list is {hot_start_list}')
    bin_hot_start_list =  hot_start_list_to_string(hot_start_list, 
                                                locations, 
                                                GRAY, 
                                                DECODING_FORMULATION
                                                )
    print(f'This is equivalent to a binary list: {bin_hot_start_list}')

    hot_start_distance = cost_fn(bin_hot_start_list)
    return hot_start_distance, bin_hot_start_list

In [ ]:
def find_best_distance(locations: int, 
                       qubits :int, 
                       cost_fn: Callable,
                       )-> tuple:
    """find the best distance from brute force by iterating through all possible routes"""
    lowest_to_date = locations*2 * 100 #to high
    lowest_list, route_list = [] , []
    for integer in range(2**qubits):
        binary_string_list = convert_integer_to_binary_list(integer, length=qubits, gray=GRAY)
        cost = cost_fn(binary_string_list)
        if cost < lowest_to_date:
            lowest_list, route_list = [] , []
            lowest_to_date = cost
            lowest_list.append(binary_string_list)
            route = convert_bit_string_to_cycle(binary_string_list, locations)
            route_list.append(route)
        elif cost == lowest_to_date:
            lowest_list.append(binary_string_list)
            route = convert_bit_string_to_cycle(binary_string_list, locations)
            route_list.append(route)
    return lowest_to_date, lowest_list, route_list

In [ ]:
def find_hamming_distance(a:list, b:list) -> int:
    """find hamming distance between two binary lists"""
    return sum(bit1 != bit2 for bit1, bit2 in zip(a, b))

In [ ]:
def find_lowest_hamming_distance(qubits: int, 
                                 lowest_list: list, 
                                 bin_hot_start_list: list,
                                 )-> int:
    """find the hamming distance between the hot start and the brute force solution"""
    lowest_hamming_distance = qubits #too high
    for routes in lowest_list:
        hamming_distance = find_hamming_distance(routes, bin_hot_start_list)
        if hamming_distance < lowest_hamming_distance:
            lowest_hamming_distance = hamming_distance
    return lowest_hamming_distance

Loop over the range of iterations and find the hot start and brute force distances and bit strings.  Find the minimum hot start distance between them:

In [ ]:
for locations in range(START_LOCATION, END_LOCATION+1):
    print(f'Processing {locations} locations.')
    qubits = find_problem_size(locations, DECODING_FORMULATION)
    distance_array, best_dist = find_distances_array(locations, print_comments=True)
    cost_fn = cost_fn_fact(locations,
                           distance_array, 
                           GRAY, 
                           method = DECODING_FORMULATION,
                           )
    ##find the hot start distance and the corresponding binary list
    hot_start_distance, bin_hot_start_list = find_hot_start_distance(distance_array, locations, cost_fn)
    quality = round(best_dist / hot_start_distance, 3)
    print(f'The hot start distance is {hot_start_distance}, compared to a best distance of {best_dist}.')
    print(f'The solution quality of the hot start is {quality:.3f}')

    #find the best distance from brute force by iterating through all possible routes
    lowest_to_date, lowest_list, route_list = find_best_distance(locations, qubits, cost_fn)
    print(f'The lowest distance found by a brute force search is {lowest_to_date}')
    print(f'The list of binary strings is {lowest_list}')
    print(f'The corresponding routes are {route_list}')

    ##find the hamming distance between the hot start and the brute force solution
    lowest_hamming_distance = find_lowest_hamming_distance(qubits, lowest_list, bin_hot_start_list)
    print(f'The lowest hamming distance is {lowest_hamming_distance}')

    new_row = {'locations': locations,  
               'qubits': qubits, 
               'best dist': best_dist,
               'hot start dist': hot_start_distance,
               'quality': quality,
               'lowest hamming dist': lowest_hamming_distance,
              }
    # Append the new row to the DataFrame
    df.loc[len(df)] = new_row


Processing 4 locations.
Reading distance data
Data will be read from filename networks\four_d.txt.
It is known that the shortest distance is 21
The hot start location list is [0, 1, 2, 3]
This is equivalent to a binary list: [0, 0, 0]
The hot start distance is 21.0, compared to a best distance of 21.
The solution quality of the hot start is 1.000
The lowest distance found by a brute force search is 21.0
The list of binary strings is [[0, 0, 0], [0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 0], [1, 1, 1]]
The corresponding routes are [[0, 1, 2, 3], [0, 1, 3, 2], [0, 2, 3, 1], [0, 3, 2, 1], [0, 1, 2, 3], [0, 1, 3, 2]]
The lowest hamming distance is 0
Processing 5 locations.
Reading distance data
Data will be read from filename networks\five_d.txt.
It is known that the shortest distance is 19
The hot start location list is [0, 3, 2, 1, 4]
This is equivalent to a binary list: [1, 0, 0, 1, 0]
The hot start distance is 21.0, compared to a best distance of 19.
The solution quality of the hot start 

print results:

In [ ]:
print(df.to_string(index=False))

 locations  qubits  best dist  hot start dist  quality  lowest hamming dist
         4       3       21.0            21.0    1.000                    0
         5       5       19.0            21.0    0.905                    2
         6       8      241.0           279.6    0.862                    5
         7      11      276.2           314.8    0.877                    5
         8      14      277.2           315.8    0.878                    8
         9      17      286.7           339.8    0.844                    9
        10      21      290.2           312.3    0.929                    8
        11      25      253.0           299.0    0.846                    9
        12      29      297.2           385.5    0.771                   10


Save file as csv:

In [9]:
filepath = Path(RESULTS_DIR).joinpath(FILENAME)
try:
    df.to_csv(filepath, index=False)
    print(f'CSV file saved successfully at {filepath}')
except Exception as e:
    print(f'An error occurred while saving the CSV: {e}')

CSV file saved successfully at results\hot_start_hamming_distance.csv
